
# Exoplanet Detection using Light Curves 

This notebook demonstrates how to process light curve data from Kepler/TESS missions, 
visualize the flux, and train a simple deep learning model to detect exoplanet transits.


In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, LSTM
from sklearn.model_selection import train_test_split


In [ ]:

# Load example Kepler light curve CSV (replace with your dataset)
# Example format: columns -> [time, flux]
data = pd.read_csv("light_curve.csv")

# Visualize
plt.figure(figsize=(10,5))
plt.plot(data['time'], data['flux'], color='blue')
plt.xlabel("Time (days)")
plt.ylabel("Flux (brightness)")
plt.title("Light Curve Example")
plt.show()


In [ ]:

# Normalize flux
flux = (data['flux'] - np.mean(data['flux'])) / np.std(data['flux'])

# Example: split into sequences
X = []
y = []  # 0 = no transit, 1 = transit (for supervised learning)
seq_len = 200

for i in range(len(flux) - seq_len):
    X.append(flux[i:i+seq_len])
    y.append(0)  # Placeholder: replace with actual labels

X = np.array(X).reshape(-1, seq_len, 1)
y = np.array(y)


In [ ]:

# Simple 1D CNN model for light curve classification
model = Sequential([
    Conv1D(16, 5, activation='relu', input_shape=(X.shape[1], 1)),
    MaxPooling1D(2),
    Conv1D(32, 5, activation='relu'),
    MaxPooling1D(2),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model (example small epochs for testing)
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=32)


In [ ]:

# Predict
preds = model.predict(X_test[:10])

print("Predictions:", preds.flatten())

# Plot first sequence example
plt.plot(X_test[0])
plt.title(f"Predicted: {preds[0][0]:.2f}")
plt.show()
